In [7]:
%pip install -q langchain_google_genai

In [21]:
from google.colab import auth
auth.authenticate_user()

In [22]:
import langchain_google_genai
import time

In [25]:
def get_embedding_with_retry(text, max_retries=5, initial_delay=1):
    """Gets an embedding with retry logic."""
    for attempt in range(max_retries):
        try:
            llm = langchain_google_genai.GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")
            embedding = llm.embed_query(text)
            return embedding
        except Exception as e:
            if "Deadline Exceeded" in str(e):
                print(f"Attempt {attempt + 1} failed: Deadline Exceeded. Retrying in {initial_delay} seconds...")
                time.sleep(initial_delay)
                initial_delay *= 2  # Exponential backoff
            else:
                print(f"Attempt {attempt + 1} failed with a different error: {e}")
                raise  # Re-raise the exception if it's not a timeout

    raise Exception("Max retries reached. Embedding failed after multiple attempts.")


In [26]:
# Example usage:
try:
    text_to_embed = "This is a sample text for embedding."
    embedding_vector = get_embedding_with_retry(text_to_embed)
    print("Embedding:", embedding_vector)
except Exception as e:
    print("Error getting embedding:", e)

Attempt 1 failed with a different error: Error embedding content: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.EmbedContent"
}
]
Error getting embedding: Error embedding content: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.EmbedContent"
}
]


**Vertex runs with no issue**

In [29]:
%pip install -qU langchain_google_vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.3 MB/s eta 0:00:00


In [30]:
from langchain_google_vertexai import VertexAIEmbeddings
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [31]:
PROJECT_ID = "langgraph-465507"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [32]:
from langchain_google_vertexai import VertexAIEmbeddings

# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [33]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

In [42]:
def get_vembedding_with_retry(text, max_retries=5, initial_delay=1):
    """Gets an embedding with retry logic."""
    for attempt in range(max_retries):
        try:
            llm = VertexAIEmbeddings(model_name="gemini-embedding-001")
            embedding = llm.embed_query(text)
            return embedding
        except Exception as e:
            if "Deadline Exceeded" in str(e):
                print(f"Attempt {attempt + 1} failed: Deadline Exceeded. Retrying in {initial_delay} seconds...")
                time.sleep(initial_delay)
                initial_delay *= 2  # Exponential backoff
            else:
                print(f"Attempt {attempt + 1} failed with a different error: {e}")
                raise  # Re-raise the exception if it's not a timeout

    raise Exception("Max retries reached. Embedding failed after multiple attempts.")


In [44]:
# Example usage:
try:
    text_to_embed = "This is a sample text for embedding."
    embedding_vector = get_vembedding_with_retry(text_to_embed)
    print("Embedding:", embedding_vector)
except Exception as e:
    print("Error getting embedding:", e)

Embedding: [0.0015431545907631516, 0.013321546837687492, -0.0053194486536085606, -0.08282909542322159, 0.005461082793772221, -0.011800210922956467, 0.00977151095867157, 0.005385535769164562, -0.001334311906248331, -0.005378674250096083, 0.012904762290418148, -0.026802031323313713, -0.01211115438491106, 0.028841877356171608, 0.11304863542318344, 0.010421168990433216, -0.004207704681903124, -0.0014325684169307351, 0.0037803505547344685, -0.010386361740529537, -0.003966893535107374, -0.009771404787898064, 0.015338588505983353, -0.0016714326338842511, 0.0017649653600528836, -0.027805721387267113, 0.017009366303682327, -0.004728352651000023, 0.03676619008183479, 0.012850195169448853, 0.004695969168096781, 0.016448263078927994, 0.03359819948673248, -0.010990782640874386, -0.004737107548862696, -0.0010081029031425714, 0.011096782982349396, 0.007046644575893879, 0.007943613454699516, 0.014006569981575012, -0.011373980902135372, -0.006569509394466877, -0.031926389783620834, 0.002691394416615367